In [4]:
# Useful starting lines
import numpy as np
from helpers import *
from methods import *
from process_data import *
from crossValidation import *

%load_ext autoreload
%autoreload 2

seed=20

# Load the dataset

In [5]:
y, tX, ids = load_csv_data('Data/train.csv')
_, tX_test, ids_test = load_csv_data('Data/test.csv')

# Preprocessing

### Umbalanced Dataset

In [6]:
higgs = np.count_nonzero(y==1)
print(f'From {y.shape[0]} training examples, {higgs} are 1, i.e. the {higgs/y.shape[0]} %')

From 250000 training examples, 85667 are 1, i.e. the 0.342668 %


IDEA: We could use Undersampling\Oversampling alghoritms

### Manage missing values

In [7]:
tX, tX_test = missing_values(tX, tX_test)

SUPERIDEA: SLIT THE DATASET IN 4 CLASSES

### Standardization

In [8]:
tX, mean_tX, std_tX = standardize(tX)
tX_test, _, _ = standardize(tX, mean_tX, std_tX)

### Other Ideas

1. New features: Apply a polynomial basis to all the X features

2. PCA, correlation analysis (scatterplot, VIF, ...), manage the 0s in the last feature

3. Outlayer analysis, leverages, cook's metric ...

In [9]:
tX, tX_test = process_data(tX, tX_test, True)

# Methods

## 1. Least Squares with Gradient Descent

In [ ]:
loss, weights = least_squares_GD(y, tX,initial_w=np.zeros(tX.shape[1]), max_iters=1000, gamma=0.005)

# it shoud be in this form
# loss, weights = least_squares_GD(y, tx, initial w, max_iters, gamma)

## 2. Least Squares with Stochastic Gradient Descent

In [ ]:
# yet to be implemented
loss, weights = least_squares_SGD(y, tX, initial_w=np.zeros(tX.shape[1]), batch_size=len(y), max_iters=1000, gamma=0.005)

## 3. Least Squares with Normal Equations 

In [68]:
loss, weights = least_squares(y, tX)

## 4. Ridge regression with Normal Equations

In [73]:
loss, weights = ridge_regression(y, tX, 0.2)

## 5. Logistic Regression with Stochastic Gradient Descent


In [ ]:
loss, weights = logistic_regression(y, tX,initial_w= np.random.random(tX.shape[1]))

## 6. Regularized Logistic Regression with Stochastic Gradient Descent

In [ ]:
# yet to be implemented
# loss, weights = reg_logistic regression(y, tX, initial w, max_iters, gamma)

# Cross Validation
IDEA: insert CV in each of the methods above

In [ ]:
# To evaluate the best lambda that minimizes the test error
loss, weights, best_lambda = cross_validation_ridge_regression(y,tX)

In [ ]:
# To evaluate the mean test error that we expect, the third argument should be one of the 6 methods used  
loss = cross_validation_general(y,tX,fun,logistic_regression,k_fold = 4)

# Prediction

#### Generate predictions and save ouput in csv format for submission

In [74]:
# Only for non logistic methods
y_pred = predict_labels(weights, tX_test)

In [64]:
# Only for Logistic methods
y_pred = sigmoid(tX@weights)
y_pred[y_pred <0.5] = -1
y_pred[y_pred > 0.5] = 1

In [75]:
OUTPUT_PATH = 'data/FIRSTsubmission.csv' 
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

ATTENTION: we are prediction too LESS 1

In [76]:
higgs = np.count_nonzero(y_pred==1)
print(f'From {y_pred.shape[0]} test examples, {higgs} are 1, i.e. the {higgs/y_pred.shape[0]} %')

From 250000 test examples, 180378 are 1, i.e. the 0.721512 %
